In [1]:
from langchain_community.chat_models import ChatOllama

from langchain_community.embeddings import OllamaEmbeddings

In [2]:
llm = ChatOllama(model="llama3:instruct", temperature=0)

In [3]:
from langchain_community.utilities import SQLDatabase

# Note: to run in Colab, you need to upload the npci.db file in the repo to the Colab folder first.
db = SQLDatabase.from_uri("sqlite:///C:/Users/Kritika/Downloads/NPCI Python LLM/npci.db", sample_rows_in_table_info=0)

def get_schema():
    return db.get_table_info()

#The purpose is to instruct the LLM write a query which will identify Bank name with highest ATM & CRM onsite number for each bank category. Instructions will be executed in following two steps:
Step 1: This will retrieve the list of banks which has highest ATM & CRM onsite number
Step 2: The output from Step 1 will be fed into the prompt to retrieve result as "Bank Category","Bank Name" & "ATM & CRM onsite" number altogether

In [4]:
# Step-1
question = "for all the 'bank category', find the 'bank name' having highest value for 'ATM & CRM on-site"
prompt = f"""Based on the table schema below, write a SQL query that would answer the user's question. Only return the SQL query with no line separator at the end.

Scheme:
{get_schema()}


Question: {question}

SQL Query:"""

print(prompt)

Based on the table schema below, write a SQL query that would answer the user's question. Only return the SQL query with no line separator at the end.

Scheme:

CREATE TABLE npci_table (
	"Bank Category" TEXT, 
	"Bank Name" TEXT, 
	"ATM & CRM Onsite" INTEGER, 
	"ATM & CRM Offsite" INTEGER, 
	"PoS" INTEGER, 
	"Micro ATM" INTEGER, 
	"Bharat QR Codes" INTEGER, 
	"UPI QR Codes" INTEGER, 
	"Credit Cards" INTEGER, 
	"Debit Cards" INTEGER, 
	"CreditCard Card Payments Transactions PoS  Volume (in actuals)" INTEGER, 
	"Credit Card Card Payments Transactions PoS  Value (in Rs 000)" INTEGER, 
	"Credit Card Card Payments Transactions Online(e-com)  Volume (in actuals)" INTEGER, 
	"Credit Card Card Payments Transactions Online(e-com)  Value (in Rs 000)" INTEGER, 
	"Credit Card Card Payments Transactions Others  Volume (in actuals)" INTEGER, 
	"Credit Card Card Payments Transactions Others  Value (in Rs 000)" INTEGER, 
	"Credit Card Cash Withdrawal At ATM Volume (in actuals)" INTEGER, 
	"Credit Card

In [5]:
answer = llm.invoke(prompt)
print(answer)

result = db.run(answer.content)
result

content='SELECT "Bank Name" FROM npci_table WHERE ("Bank Category", "ATM & CRM Onsite") IN (SELECT "Bank Category", MAX("ATM & CRM Onsite") FROM npci_table GROUP BY "Bank Category");' response_metadata={'model': 'llama3:instruct', 'created_at': '2024-08-11T19:46:18.5379756Z', 'message': {'role': 'assistant', 'content': ''}, 'done_reason': 'stop', 'done': True, 'total_duration': 73878950100, 'load_duration': 133849500, 'prompt_eval_count': 529, 'prompt_eval_duration': 9471937000, 'eval_count': 49, 'eval_duration': 64254195000} id='run-43ef8f2e-6b6d-43ec-8f4a-bed1bbeccbc2-0'


"[('STATE BANK OF INDIA',), ('HDFC BANK LTD',), ('DBS INDIA BANK LTD',), ('PAYTM PAYMENTS BANK',), ('ESAF SMALL FINANCE BANK LTD',)]"

In [6]:
#Step 2
question = "for each bank in the list [('DBS INDIA BANK LTD',), ('PAYTM PAYMENTS BANK',), ('HDFC BANK LTD',), ('STATE BANK OF INDIA',), ('ESAF SMALL FINANCE BANK LTD',)] find its bank category and ATM & CRM Onsite"
prompt = f"""Based on the table schema below, write a SQL query that would answer the user's question. Only return the SQL query with no line separator at the end.

Scheme:
{get_schema()}

Question: {question}

SQL Query:"""

print(prompt)

Based on the table schema below, write a SQL query that would answer the user's question. Only return the SQL query with no line separator at the end.

Scheme:

CREATE TABLE npci_table (
	"Bank Category" TEXT, 
	"Bank Name" TEXT, 
	"ATM & CRM Onsite" INTEGER, 
	"ATM & CRM Offsite" INTEGER, 
	"PoS" INTEGER, 
	"Micro ATM" INTEGER, 
	"Bharat QR Codes" INTEGER, 
	"UPI QR Codes" INTEGER, 
	"Credit Cards" INTEGER, 
	"Debit Cards" INTEGER, 
	"CreditCard Card Payments Transactions PoS  Volume (in actuals)" INTEGER, 
	"Credit Card Card Payments Transactions PoS  Value (in Rs 000)" INTEGER, 
	"Credit Card Card Payments Transactions Online(e-com)  Volume (in actuals)" INTEGER, 
	"Credit Card Card Payments Transactions Online(e-com)  Value (in Rs 000)" INTEGER, 
	"Credit Card Card Payments Transactions Others  Volume (in actuals)" INTEGER, 
	"Credit Card Card Payments Transactions Others  Value (in Rs 000)" INTEGER, 
	"Credit Card Cash Withdrawal At ATM Volume (in actuals)" INTEGER, 
	"Credit Card

In [7]:
answer = llm.invoke(prompt)
print(answer)

result = db.run(answer.content)
result

content='SELECT "Bank Category", "Bank Name", "ATM & CRM Onsite"\nFROM npci_table\nWHERE "Bank Name" IN (\'DBS INDIA BANK LTD\', \'PAYTM PAYMENTS BANK\', \'HDFC BANK LTD\', \'STATE BANK OF INDIA\', \'ESAF SMALL FINANCE BANK LTD\')' response_metadata={'model': 'llama3:instruct', 'created_at': '2024-08-11T19:48:32.8468318Z', 'message': {'role': 'assistant', 'content': ''}, 'done_reason': 'stop', 'done': True, 'total_duration': 65862506700, 'load_duration': 84079500, 'prompt_eval_count': 564, 'prompt_eval_duration': 24434395000, 'eval_count': 67, 'eval_duration': 41332108000} id='run-92ccbf7a-2c01-4d0b-97c4-13c557c3abb3-0'


"[('Public Sector Banks', 'STATE BANK OF INDIA', 25232), ('Private Sector Banks', 'HDFC BANK LTD', 11395), ('Foreign Banks', 'DBS INDIA BANK LTD', 434), ('Payment Banks', 'PAYTM PAYMENTS BANK', 1), ('Small Finance Banks', 'ESAF SMALL FINANCE BANK LTD', 605)]"

In [8]:
#query 2
question = "Which bank has the highest Credit Card Card Payments Transactions Online(e-com)?"
prompt = f"""Based on the table schema below, write a SQL query that would answer the user's question. Only return the SQL query with no line separator at the end.

Scheme:
{get_schema()}

Question: {question}

SQL Query:"""

print(prompt)

Based on the table schema below, write a SQL query that would answer the user's question. Only return the SQL query with no line separator at the end.

Scheme:

CREATE TABLE npci_table (
	"Bank Category" TEXT, 
	"Bank Name" TEXT, 
	"ATM & CRM Onsite" INTEGER, 
	"ATM & CRM Offsite" INTEGER, 
	"PoS" INTEGER, 
	"Micro ATM" INTEGER, 
	"Bharat QR Codes" INTEGER, 
	"UPI QR Codes" INTEGER, 
	"Credit Cards" INTEGER, 
	"Debit Cards" INTEGER, 
	"CreditCard Card Payments Transactions PoS  Volume (in actuals)" INTEGER, 
	"Credit Card Card Payments Transactions PoS  Value (in Rs 000)" INTEGER, 
	"Credit Card Card Payments Transactions Online(e-com)  Volume (in actuals)" INTEGER, 
	"Credit Card Card Payments Transactions Online(e-com)  Value (in Rs 000)" INTEGER, 
	"Credit Card Card Payments Transactions Others  Volume (in actuals)" INTEGER, 
	"Credit Card Card Payments Transactions Others  Value (in Rs 000)" INTEGER, 
	"Credit Card Cash Withdrawal At ATM Volume (in actuals)" INTEGER, 
	"Credit Card

In [9]:
answer = llm.invoke(prompt)
print(answer)

result = db.run(answer.content)
result

content='SELECT "Bank Name", "Credit Card Card Payments Transactions Online(e-com)  Value (in Rs 000)"\nFROM npci_table\nORDER BY "Credit Card Card Payments Transactions Online(e-com)  Value (in Rs 000)" DESC\nLIMIT 1;' response_metadata={'model': 'llama3:instruct', 'created_at': '2024-08-11T19:49:55.7446812Z', 'message': {'role': 'assistant', 'content': ''}, 'done_reason': 'stop', 'done': True, 'total_duration': 40807212400, 'load_duration': 273164900, 'prompt_eval_count': 518, 'prompt_eval_duration': 8780381000, 'eval_count': 55, 'eval_duration': 31654869000} id='run-cf0abab8-7f1a-4583-991c-e9db473c71e4-0'


"[('HDFC BANK LTD', 279556606)]"